In [1]:
import os
import numpy as np
import pandas as pd

import torch
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
from nltk.tokenize import word_tokenize

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("/kaggle/input/aeon-essays-dataset/essays.csv")
df

,title,description,essay,authors,source_url,thumbnail_url
0,Space exploration,When self-replicating craft bring life to the ...,"Some time late this century, someone will push...",Jay Olson,https://aeon.co//essays/cosmic-expansion-is-a-...,https://images.aeonmedia.co/images/9239658f-b9...
1,History of science,"To the detriment of the public, scientists and...",Would boycotting Russian scientists be an effe...,Lorraine Daston & Peter Harrison,https://aeon.co//essays/science-and-history-ca...,https://images.aeonmedia.co/images/7e9ea9e3-03...
2,Religion,"Once a centre of Afghan culture, Sufism seems ...",My introduction into the world of Afghanistan’...,Annika Schmeding,https://aeon.co//essays/sufi-transitions-betwe...,https://images.aeonmedia.co/images/957fb6c9-40...
3,Thinkers and theories,The intrepid logician Kurt Gödel believed in t...,"As the foremost logician of the 20th century, ...",Alexander T Englert,https://aeon.co//essays/kurt-godel-his-mother-...,https://images.aeonmedia.co/images/cbe24f46-98...
4,Thinkers and theories,"For Rachel Bespaloff, philosophy was a sensual...",Shortly after Rachel Bespaloff’s suicide in 19...,Isabel Jacobs,https://aeon.co//essays/for-rachel-bespaloff-p...,https://images.aeonmedia.co/images/536e31b1-dc...
...,...,...,...,...,...,...
2230,Architecture,We can build structures that last for centurie...,Make a model of the world in your mind. Popula...,Marek Kohn,https://aeon.co//essays/who-do-we-care-about-w...,https://images.aeonmedia.co/images/1dc63341-54...
2231,Human rights and justice,The ruling that Anders Breivik is sane leaves ...,Anders Behring Breivik has been declared sane ...,Stephen Cave,https://aeon.co//essays/could-the-law-make-us-...,https://images.aeonmedia.co/images/8f0c3d57-bd...
2232,Demography and migration,"Despite its turmoil, ever more people are risk...",It’s 9.30pm and I am in a police station in Di...,Helena Drysdale,https://aeon.co//essays/greeces-other-crisis-c...,https://images.aeonmedia.co/images/71629b3c-0e...
2233,Ecology and environmental sciences,We picture ancient Britain as a land of enchan...,"As part of a recent walk across England, I ent...",Hugh Thomson,https://aeon.co//essays/who-chopped-down-brita...,https://images.aeonmedia.co/images/5892900e-30...


In [3]:
df = df[["description", "essay"]]
df

,description,essay
0,When self-replicating craft bring life to the ...,"Some time late this century, someone will push..."
1,"To the detriment of the public, scientists and...",Would boycotting Russian scientists be an effe...
2,"Once a centre of Afghan culture, Sufism seems ...",My introduction into the world of Afghanistan’...
3,The intrepid logician Kurt Gödel believed in t...,"As the foremost logician of the 20th century, ..."
4,"For Rachel Bespaloff, philosophy was a sensual...",Shortly after Rachel Bespaloff’s suicide in 19...
...,...,...
2230,We can build structures that last for centurie...,Make a model of the world in your mind. Popula...
2231,The ruling that Anders Breivik is sane leaves ...,Anders Behring Breivik has been declared sane ...
2232,"Despite its turmoil, ever more people are risk...",It’s 9.30pm and I am in a police station in Di...
2233,We picture ancient Britain as a land of enchan...,"As part of a recent walk across England, I ent..."


In [4]:
texts = np.hstack([df["description"], df["essay"]])
texts = texts.tolist()

In [5]:
len(texts)

4470

In [6]:
# No row has null value
for text in texts:
    if len(text) < 100:
        print(text)

The transgressions of working-class women formed the revolutionary heart of the 1871 Paris Commune
The ultimate value of the world can be discovered if you are sensitive to what is beautiful
Corruption is a truly global crisis and the wealth addiction that feeds it is hiding in plain sight
Does the existence of a multiverse hold the key for why nature’s laws seem so simple?
The terms ‘mind’ and ‘mental’ are messy, harmful and distracting. We should get rid of them
What pea-sized brain organoids reveal about consciousness, the self and our future as a species
Saturated with rites and symbols, psychology feeds a deep human need once nourished by mythology
The casualties of the First World War brought a new understanding of human fragility and wholeness
Given how little control we have of our wandering minds, how can we cultivate real mental autonomy?
Is evolutionary science due for a major overhaul – or is talk of ‘revolution’ misguided?
You are only as strong as your powers of attention

In [7]:
for i, text in enumerate(texts):
    text = text.split()
    if len(text) > 200:
        text = text[:100]
        text = " ".join(text)
        texts[i] = text

In [8]:
string = 'Some time late this century, someone will push a button, unleashing a life force on the cosmos. Within 1,000 years, every star you can see at night will host intelligent life.'
string = string.split()
string = " ".join(string[:10])
string

'Some time late this century, someone will push a button,'

In [9]:
for text in texts:
    if len(text.split()) > 300:
        print(text)

In [10]:
conversations = texts
del texts

In [11]:
import re

def remove_unwanted_characters(x):
    x = x.replace(u'\xa0', u' ')
    x = x.replace('\u200a',' ')
    x = re.sub(r'[\t\n]', '', x)
    x = re.sub(r'<.*?>', '', x)
    x = re.sub(r'\s+', ' ', x).strip()
    return x

In [12]:
for i in range(len(conversations)):
    conversations[i] = remove_unwanted_characters(conversations[i])
conversations[100]

'By day an insurance official, by night he was an incessant, insomniacal scribe of the space between waking and dreaming'

In [13]:
len(conversations)

4470

In [14]:
tokenized_conversations = []
import time
start = time.time()
for i, conversation in enumerate(conversations):
    tokenized_conversations.append(word_tokenize(conversation))
    if (i + 1) % 100 == 0:
        time_taken = time.time() - start
        print(f"{i+1}th Sentence. {time_taken:.2f}s time taken total. {(time_taken / 100) * (len(conversations) - i): .2f} Time left")
        start = time.time()

100th Sentence. 0.04s time taken total.  1.91 Time left
200th Sentence. 0.03s time taken total.  1.27 Time left
300th Sentence. 0.03s time taken total.  1.20 Time left
400th Sentence. 0.03s time taken total.  1.20 Time left
500th Sentence. 0.03s time taken total.  1.15 Time left
600th Sentence. 0.03s time taken total.  1.10 Time left
700th Sentence. 0.03s time taken total.  1.10 Time left
800th Sentence. 0.03s time taken total.  1.06 Time left
900th Sentence. 0.03s time taken total.  1.13 Time left
1000th Sentence. 0.03s time taken total.  1.11 Time left
1100th Sentence. 0.04s time taken total.  1.22 Time left
1200th Sentence. 0.03s time taken total.  1.09 Time left
1300th Sentence. 0.03s time taken total.  1.06 Time left
1400th Sentence. 0.03s time taken total.  1.02 Time left
1500th Sentence. 0.03s time taken total.  0.88 Time left
1600th Sentence. 0.03s time taken total.  0.98 Time left
1700th Sentence. 0.03s time taken total.  0.87 Time left
1800th Sentence. 0.03s time taken total.

In [15]:
tokenized_conversations[0][:100]

['When',
 'self-replicating',
 'craft',
 'bring',
 'life',
 'to',
 'the',
 'far',
 'Universe',
 ',',
 'a',
 'religious',
 'cult',
 ',',
 'not',
 'science',
 ',',
 'is',
 'likely',
 'to',
 'be',
 'the',
 'driving',
 'force']

In [16]:
word_to_id = {word: idx for idx, word in enumerate(set(word for sentence in tokenized_conversations for word in sentence))}
id_to_word = {idx: word for word, idx in word_to_id.items()}

In [17]:
len(word_to_id)

29523

In [18]:
len(id_to_word)

29523

In [19]:
tokenized_conversation_ids = [[word_to_id[word] for word in sentence] for sentence in tokenized_conversations]

In [20]:
tokenized_conversation_ids[0][:100]

[25120,
 28,
 25227,
 2662,
 5708,
 17331,
 11975,
 27715,
 13388,
 11502,
 14993,
 11406,
 2680,
 11502,
 27228,
 20616,
 11502,
 26104,
 27419,
 17331,
 13497,
 11975,
 22696,
 12135]

In [21]:
len(tokenized_conversation_ids)

4470

In [22]:
def generate_padded_sequences(tokenized_conversation_ids, batch_size=100):
    input_sequences = []

    for idx, sentence_ids in enumerate(tokenized_conversation_ids):
        for i in range(1, len(sentence_ids)):
            n_gram_sequence = sentence_ids[:i+1]
            input_sequences.append(torch.tensor(n_gram_sequence))

        if (idx + 1) % batch_size == 0:
            padded_sequences = pad_sequence(input_sequences, batch_first=True, padding_value=0)
            yield padded_sequences
            input_sequences = []

    if input_sequences:
        padded_sequences = pad_sequence(input_sequences, batch_first=True, padding_value=0)
        yield padded_sequences


In [23]:
input_sequences = []
start = time.time()
for idx, sentence_ids in enumerate(tokenized_conversation_ids):
    for i in range(1, len(sentence_ids)):
        n_gram_sequence = sentence_ids[:i+1]
        input_sequences.append(n_gram_sequence)
    if (idx+1 % 100) == 0:
        time_taken = time.time() - start
        print(f"{idx+1} Done. {time_taken:.2f}s Total Time. {(time_taken/100)*(len(tokenized_conversation_ids - idx)):.2f}s Time Remaining.")
        start = time.time()
        
print("Total input sequences: ", len(input_sequences))

Total input sequences:  305741


In [24]:
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

174

In [25]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [26]:
input_sequences[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [27]:
vocab = word_to_id


In [28]:
for key, value in vocab.items():
    if value == 10193:
        print(key)

25


In [29]:
from torch.utils.data import Dataset, DataLoader

import torch
from torch.utils.data import Dataset

class NextWordDataset(Dataset):
    def __init__(self, input_sequences, vocab):
        self.input_sequences = input_sequences
        self.vocab_size = len(vocab)

    def __len__(self):
        return len(self.input_sequences)

    def __getitem__(self, idx):
        input_sequence = self.input_sequences[idx]

        input_word_sequence = input_sequence[:-1]
        input_tensor = torch.tensor(input_word_sequence)

        target_word = input_sequence[-1]
        target_tensor = torch.zeros(self.vocab_size)
        target_tensor[target_word] = 1

        return input_tensor, target_tensor

dataset = NextWordDataset(input_sequences, vocab)


In [30]:
class NextWordPredictor(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(NextWordPredictor, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input):
        embedded = self.embedding(input)
        output, _ = self.lstm(embedded)
        output = self.linear(output)
        return output


In [31]:
vocab_size = len(set(word for sentence in tokenized_conversations for word in sentence))
vocab_size

29523

In [32]:
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
for input, targets in dataloader:
    print(input.shape)
    break

torch.Size([64, 173])


In [33]:
len(dataloader)

4778

In [34]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [36]:
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
len(dataloader)

4778

In [ ]:
import time

start = time.time()

embedding_dim = 50
hidden_dim = 100
vocab_size = vocab_size

epochs = 5

model = NextWordPredictor(vocab_size, embedding_dim, hidden_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

for epoch in range(epochs):
    for i, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.long())
        loss.backward()
        optimizer.step()
        if i == 0:
            print(f"{epoch + 1} Epoch Started.\n")
        if (i + 1) % 100 == 0:
            time_taken = time.time() - start
            print(f"Epoch:{epoch+1}, Step: {i+1}: Loss: {loss:.2f}, Time Taken: {time_taken:.2f}s\n")
            start = time.time()



1 Epoch Started.

Epoch:1, Step: 100: Loss: 1.28, Time Taken: 17.05s

Epoch:1, Step: 200: Loss: 0.94, Time Taken: 17.06s

Epoch:1, Step: 300: Loss: 0.89, Time Taken: 17.04s

Epoch:1, Step: 400: Loss: 0.88, Time Taken: 16.93s

Epoch:1, Step: 500: Loss: 0.86, Time Taken: 16.89s

Epoch:1, Step: 600: Loss: 0.86, Time Taken: 16.91s

Epoch:1, Step: 700: Loss: 0.85, Time Taken: 16.98s

Epoch:1, Step: 800: Loss: 0.85, Time Taken: 16.94s

Epoch:1, Step: 900: Loss: 0.84, Time Taken: 16.95s

Epoch:1, Step: 1000: Loss: 0.84, Time Taken: 16.93s

Epoch:1, Step: 1100: Loss: 0.83, Time Taken: 16.93s

Epoch:1, Step: 1200: Loss: 0.83, Time Taken: 16.93s

Epoch:1, Step: 1300: Loss: 0.83, Time Taken: 16.94s

Epoch:1, Step: 1400: Loss: 0.83, Time Taken: 16.93s

Epoch:1, Step: 1500: Loss: 0.83, Time Taken: 16.91s



In [ ]:
PATH = "saved_model.pth"
torch.save(model, PATH)